In [ ]:
{
  "nbformat": 4,
  "nbformat_minor": 5,
  "metadata": {
    "kernelspec": {
      "name": "python3",
      "display_name": "Python 3"
    },
    "language_info": {
      "name": "python",
      "version": "3.x"
    }
  },
  "cells": [
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "# MASWE Failure Analysis Notebook\n",
        "\n",
        "This notebook helps you inspect agent logs saved under\n",
        "`/app/workspace/agent_logs/<run_id>/`."
      ]
    },
    {
      "cell_type": "code",
      "metadata": {},
      "source": [
        "import json\n",
        "import os\n",
        "from pathlib import Path\n",
        "from collections import Counter\n",
        "\n",
        "import matplotlib.pyplot as plt\n",
        "import pandas as pd\n",
        "\n",
        "LOG_ROOT = Path(\"/app/workspace/agent_logs\")\n",
        "RUN_ID = os.environ.get(\"MASWE_ANALYSIS_RUN_ID\", \"YOUR_RUN_ID_HERE\")\n",
        "RUN_DIR = LOG_ROOT / RUN_ID\n",
        "RUN_DIR"
      ],
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "code",
      "metadata": {},
      "source": [
        "def read_jsonl(path: Path):\n",
        "    if not path.exists():\n",
        "        return []\n",
        "    rows = []\n",
        "    with path.open(\"r\", encoding=\"utf-8\") as f:\n",
        "        for line in f:\n",
        "            line = line.strip()\n",
        "            if not line:\n",
        "                continue\n",
        "            try:\n",
        "                rows.append(json.loads(line))\n",
        "            except json.JSONDecodeError:\n",
        "                pass\n",
        "    return rows\n",
        "\n",
        "llm_calls = read_jsonl(RUN_DIR / \"llm_calls.jsonl\")\n",
        "trace = json.load((RUN_DIR / \"trace.json\").open(\"r\", encoding=\"utf-8\")) if (RUN_DIR / \"trace.json\").exists() else []\n",
        "\n",
        "len(llm_calls), len(trace)"
      ],
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "code",
      "metadata": {},
      "source": [
        "df_llm = pd.DataFrame(llm_calls)\n",
        "df_trace = pd.DataFrame(trace)\n",
        "df_llm.head()"
      ],
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "code",
      "metadata": {},
      "source": [
        "# Simple timeline of LLM calls by role\n",
        "if not df_llm.empty:\n",
        "    df_llm_sorted = df_llm.sort_values(\"ts\")\n",
        "    df_llm_sorted[\"idx\"] = range(len(df_llm_sorted))\n",
        "\n",
        "    plt.figure(figsize=(10, 4))\n",
        "    for role, grp in df_llm_sorted.groupby(\"role\"):\n",
        "        plt.plot(grp[\"idx\"], grp.get(\"completion_tokens\", 1), marker=\"o\", linestyle=\"-\", label=role)\n",
        "\n",
        "    plt.xlabel(\"Call index\")\n",
        "    plt.ylabel(\"Estimated completion tokens\")\n",
        "    plt.title(f\"LLM calls over time for run {RUN_ID}\")\n",
        "    plt.legend()\n",
        "    plt.tight_layout()\n",
        "    plt.show()"
      ],
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "code",
      "metadata": {},
      "source": [
        "# Very coarse \"failure clustering\": look at most common failing messages or events\n",
        "failure_events = [e for e in trace if e.get(\"type\") in {\"env_message\", \"humaneval_problem_done\"}]\n",
        "\n",
        "summaries = []\n",
        "for ev in failure_events:\n",
        "    data = ev.get(\"data\", {})\n",
        "    # for HumanEval we track pass_at_3 and sample_results\n",
        "    if ev[\"type\"] == \"humaneval_problem_done\":\n",
        "        if data.get(\"pass_at_3\", 1) < 1:\n",
        "            summaries.append(f\"HE_FAIL::{data.get('task_id')}::pass_at_3={data.get('pass_at_3')}\")\n",
        "    else:\n",
        "        # generic env_message – bucket by short repr\n",
        "        msg = str(data.get(\"repr\", \"\"))[:80]\n",
        "        summaries.append(msg)\n",
        "\n",
        "Counter(summaries).most_common(10)"
      ],
      "execution_count": null,
      "outputs": []
    }
  ]
}
